In [1]:
from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import subprocess
import soundfile as sf

In [2]:
files = glob('malaysian-podcast_processed/**/*/*.json', recursive = True)
files.extend(glob('/home/husein/ssd3/sg-podcast_processed/**/*/*.json', recursive = True))

len(files)

22492

In [3]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [4]:
threshold = -18

def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def new_path_lang(f):
    f = f.replace('.mp3', '.language')
    f = f.replace('_processed/', '_processed_language/')
    return f

def chunk(alignment, reject = threshold, minimum_length = 2.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls
        
def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string.lower()

def detect_extra(word):
    word = clean(word)
    return word in [
        'eh', 'ehh', 'oh', 'ohh', 'uhm', 'uhmm',
        'um', 'em', 'emm', 'ah', 'ha', 'ok', 'okay',
        'so', 'yes', 'no', 'ah', 'aa', 'so', 'uh', 'um', 'eh', 'ha', 'oh', 'ye', 'haa',
        'oi', 'ya', 'leh', 'lah', 'haiya', 'hoi', 'haha', 'hahaha',
        'then', 'it s'
    ]

from collections import defaultdict

def generate_trigrams(text):
    words = text.split()
    return list(zip(words, words[1:], words[2:]))

def skip_trigrams(text):
    trigrams = generate_trigrams(text)
    count = defaultdict(int)
    total = 0
    for t in trigrams:
        count[''.join(t)] += 1
        total += 1
    if len(count.keys()) < 3:
        return True
    for k, v in count.items():
        if (v / total) > 0.2:
            return True
    return False

In [5]:
!rm -rf podcast-chunk
!mkdir podcast-chunk

In [6]:
def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
                
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            for a in align:
                if detect_extra(a['text']):
                    a['score'] = 0.0
                
            scores = [a for a in align if a['score'] <= threshold]
            if not len(scores):
                continue
            
            chunks = chunk(align)
            if len(chunks):
                y, sr = sf.read(audio_path)
                for no, c in enumerate(chunks):
                    if len(c) == len(align):
                        continue
                    try:
                        
                        words = [c_['text'] for c_ in c if len(c_['text']) <= 1]
                        if (len(words) / len(c)) > 0.5:
                            print(c)
                            continue
                        
                        skip = False
                        
                        for c_ in c:
                            if (c_['end'] - c_['start']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                        
                        for i in range(1, len(c), 1):
                            if (c[i]['start'] - c[i - 1]['end']) >= 2:
                                skip = True
                                break
                        if skip:
                            continue
                            
                        t = ' '.join([c_['text'] for c_ in c])
                        start = c[0]['start']
                        end = c[-1]['end']
                        
                        a = audio_path.replace('/', '_').replace('.mp3', '') 
                        a = os.path.join('podcast-chunk', f'{a}_{no}.mp3')
                        if not os.path.exists(a):
                            sf.write(a, y[int(sr * start): int(sr * end)], sr)

                        data.append({
                            'audio': a,
                            'transcription': t,
                        })
                    except:
                        pass
    
    return data

In [7]:
d = loop((files[-1:], 0))

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2334.06it/s]


In [8]:
len(d)

0

In [ ]:
import IPython.display as ipd
ipd.Audio(d[-1]['audio'])

In [10]:
data = mp.multiprocessing(files, loop, cores = 15)

 20%|█████████████████▏                                                                    | 299/1499 [03:53<56:16,  2.81s/it]

[{'start': 0.08, 'end': 0.08, 'text': 'I', 'score': 0.0}, {'start': 0.14, 'end': 0.14, 'text': 'P', 'score': 0.0}, {'start': 0.3, 'end': 0.3, 'text': 'T', 'score': 0.0}, {'start': 0.54, 'end': 0.54, 'text': 'A', 'score': 0.0}, {'start': 0.7, 'end': 0.86, 'text': 'atau', 'score': -0.5529270172119141}, {'start': 1.02, 'end': 1.02, 'text': 'I', 'score': 0.0}, {'start': 1.1, 'end': 1.1, 'text': 'P', 'score': 0.0}, {'start': 1.22, 'end': 1.22, 'text': 'T', 'score': 0.0}, {'start': 1.56, 'end': 1.56, 'text': 'S', 'score': 0.0}, {'start': 1.8, 'end': 1.96, 'text': 'untuk', 'score': -0.2971644401550293}, {'start': 2.02, 'end': 2.22, 'text': 'pulang', 'score': -0.07382869720458984}, {'start': 2.26, 'end': 2.3, 'text': 'ke', 'score': -0.020725250244140625}, {'start': 2.38, 'end': 2.76, 'text': 'kampung', 'score': -0.22738373279571533}, {'start': 2.86, 'end': 2.96, 'text': 'dan', 'score': -0.011804580688476562}, {'start': 3.14, 'end': 3.94, 'text': 'berkemungkinan', 'score': -1.3972400426864624}]

 89%|███████████████████████████████████████████████████████████████████████████▌         | 1332/1499 [18:41<00:35,  4.65it/s]

[{'start': 0.02, 'end': 0.02, 'text': 'I', 'score': 0.0}, {'start': 0.1, 'end': 0.24, 'text': 'think,', 'score': -17.1785888671875}, {'start': 1.66, 'end': 2.02, 'text': 'I', 'score': 0.0}]


100%|█████████████████████████████████████████████████████████████████████████████████████| 1499/1499 [21:31<00:00,  1.16it/s]


In [11]:
len(data)

205831

In [12]:
from collections import defaultdict

uniques = defaultdict(int)
for d in tqdm(data):
    uniques[d['audio'].split('_processed')[0]] += 1
    
uniques

100%|████████████████████████████████████████████████████████████████████████████| 205831/205831 [00:00<00:00, 3081890.96it/s]


defaultdict(int,
            {'podcast-chunk/malaysian-podcast': 166913,
             'podcast-chunk/_home_husein_ssd3_sg-podcast': 38918})

In [13]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,audio,transcription
0,podcast-chunk/malaysian-podcast_processed_3 Te...,"Ada satu pepatah dalam MMA ni kata, don't give..."
1,podcast-chunk/malaysian-podcast_processed_3 Te...,"di dalam, you know, sukan grappling ataupun MM..."
2,podcast-chunk/malaysian-podcast_processed_3 Te...,"Tapi yang common orang buat ni biasa, rear nak..."
3,podcast-chunk/malaysian-podcast_processed_3 Te...,"Rear Naked Choke kot. Sebab, bila opponent"
4,podcast-chunk/malaysian-podcast_processed_3 Te...,"tu berada di belakang apa, posisi, maksudnya b..."


In [14]:
df.to_parquet('verify-text-chunk-podcasts.parquet')

In [15]:
!du -hs new_chunk

8.1G	new_chunk


In [16]:
# !zip -rq text-chunk-podcasts.zip new_chunk

In [17]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="verify-text-chunk-podcasts.parquet",
    path_in_repo="data/podcasts_chunk-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)


verify-text-chunk-podcasts.parquet:   0%|          | 0.00/16.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/c5a7937e39dff59f1233e47ece68de758add7269', commit_message='Upload data/podcasts_chunk-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='c5a7937e39dff59f1233e47ece68de758add7269', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)